In [7]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data = data.iloc[start_index+3:end_index-1,:]
    data.columns = columns
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            resultados.append({
                'Símbolo': fila_in['Símbolo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': fila_out['Beneficio'] - fila_in['Comisión'] - fila_out['Comisión']
            })
    return pd.DataFrame(resultados)

In [9]:
filename='ReportTester-4000009439.xlsx'
df = load_xls_report(filename)
df.head()

,Símbolo,Precio Entrada,Precio Salida,Volumen,Beneficio
0,GDAXI,15920.8,15886.1,0.01,3.53
1,GDAXI,15865.6,15870.8,0.01,0.58
2,GDAXI,15877.3,15881.1,0.01,-0.32
3,GDAXI,15874.3,15873.6,0.01,0.13
4,GDAXI,15882.3,15873.6,0.01,0.93


In [30]:
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean()
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean()
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades

In [35]:
print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")

Número de operaciones totales: 14929
Número de operaciones ganadoras: 9752
Número de operaciones perdedoras: 5177
Media de puntos en operaciones ganadoras 7.31 puntos
Media de puntos en operaciones perdedoras 15.58 puntos
Ganancia media: 0.79€
Pérdida media: -1.50€
Rentabilidad esperada: -0.0036€
Porcentaje de aciertos actual: 65.32%
Porcentaje de aciertos para equilibrio: 65.48%
